# Define Imports and Set Up GPU

In [1]:
import tensorflow as tf
import numpy as np
from tqdm import tqdm
import os
#from skimage.transform import resize
#import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Set Up Normalization

In [3]:
def norma2(input_data):
    """Normalize all data based on their own max values
    
    Parameters
    ==========
    
    input_data - (np.array)
        the full array of all 2D images to normalize based upon their own np.max value
        
        
    Returns
    =======
    The normalized data
    
    """
    
    maxies = np.max(input_data, axis=(1,2))
    if np.count_nonzero(maxies) != len(input_data):
        print('Error - Divide By Zero Found')
    output = np.divide(input_data, maxies[:,None, None])
    
    return output

# Import Trained Model

In [2]:
# Trained on all Augmented Data after AutoKeras created network
model = tf.keras.models.load_model('../trained_model-Autokeras/')

In [3]:
# Only trained on Edge and Screw Defects - 9 epochs through AutoKeras
model = tf.keras.models.load_model('../untrained_model-Autokeras/')

# Test Model On Testing Data

In [6]:
# Load in the already split Training and Test Set2
for i in ['2', '3', '14']:

    xTest_testing = np.load(f'../testing_data/noise_test_c{i}_dataset_01.npy', allow_pickle=True)
    yTest_testing = np.load(f'../testing_data/noise_test_c{i}_labels_01.npy', allow_pickle=True)

    # For padding arrays
    value = 0

    # Section out Perfect - Edge - Screw - Testing Testing - C2 Data
    perf_dat = []
    edge_dat = []
    screw_dat = []

    perf_lab = []
    edge_lab = []
    screw_lab = []


    #for idx, labels in tqdm(enumerate(yTest_testing), total=len(yTest_testing)):
    for idx, labels in enumerate(yTest_testing):
        if labels[0] == 1:
            perf_lab.append(labels)
            perf_dat.append(xTest_testing[idx])

        elif labels[1] == 1:
            edge_lab.append(labels)
            edge_dat.append(xTest_testing[idx])

        elif labels[2] == 1:
            screw_lab.append(labels)
            screw_dat.append(xTest_testing[idx])

    perf_dat = np.asarray(perf_dat)
    edge_dat = np.asarray(edge_dat)
    screw_dat = np.asarray(screw_dat)

    perf_lab = np.asarray(perf_lab)
    edge_lab = np.asarray(edge_lab)
    screw_lab = np.asarray(screw_lab)



    X_test_testing = np.vstack((perf_dat, edge_dat,screw_dat))
    y_test_testing = np.vstack((perf_lab, edge_lab, screw_lab))

    y_test = np.asarray(y_test_testing)

    # Normalize
    X_test_testing = norma2(X_test_testing)


    X_test_testing = X_test_testing.reshape((X_test_testing.shape[0], 64, 64, 1))

    X_test_testing = np.asarray(np.repeat(X_test_testing, 3, 3))


    actual_vals = []

    for v in y_test:
        if all(v == [1,0,0]):
            actual_vals.append(0)

        elif all(v == [0,1,0]):
            actual_vals.append(1)

        elif all(v == [0,0,1]):
            actual_vals.append(2)


    preds = model.predict(X_test_testing)

    from sklearn.metrics import confusion_matrix

    pred_vals = []
    for v2 in preds:
        pred_vals.append(np.argmax(v2))


    def percents(conf):
        ars = []
        for idx, ar in enumerate(conf):
            total = np.sum(ar)
            output = ar[idx]/total
            ars.append(output)
        return ars

    
    conf = confusion_matrix(actual_vals, pred_vals, labels= (0, 1, 2))
    print(conf)
    coos = percents(conf)
    print(coos)
    print('')

[[3456    0    0]
 [   0 3347  109]
 [   0  457 2999]]
[1.0, 0.9684606481481481, 0.8677662037037037]

[[1728    0    0]
 [   0 1626  102]
 [   0  142 1586]]
[1.0, 0.9409722222222222, 0.9178240740740741]

[[1728    0    0]
 [   1 1598  129]
 [   0   29 1699]]
[1.0, 0.9247685185185185, 0.9832175925925926]

